In [30]:
import pandas as pd
import tensorflow.keras.backend as K
from tensorflow.keras import Input, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [31]:
def build_lstm_model(input_shape=(20, 1)):
    model = Sequential()
    model.add(Input(shape=input_shape))       

    model.add(LSTM(64, activation='tanh', return_sequences=False))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(1))
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', regression_accuracy]
    )
    return model

def regression_accuracy(y_true, y_pred):
    return K.mean(K.less_equal(K.abs(y_true - y_pred), 1.0))

In [32]:
#500 temperaturas generadas aleatoriamente
np.random.seed(42)
temperaturas = np.cumsum(np.random.randn(500) * 0.5 + 0.1) + 20

scaler = MinMaxScaler()
temperaturas_norm = scaler.fit_transform(temperaturas.reshape(-1, 1)).flatten()

X, y = [], []
for i in range(len(temperaturas_norm) - 20):
    X.append(temperaturas_norm[i:i+20])
    y.append(temperaturas_norm[i+20])

X = np.array(X)
y_train = np.array(y)
X_train = X.reshape((X.shape[0], X.shape[1], 1))

In [33]:
model = build_lstm_model(input_shape=(20, 1))  
model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - loss: 1.8352 - mae: 1.0825 - regression_accuracy: 0.5204 - val_loss: 0.7246 - val_mae: 0.8502 - val_regression_accuracy: 1.0000
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.2972 - mae: 0.8905 - regression_accuracy: 0.6175 - val_loss: 0.6277 - val_mae: 0.7912 - val_regression_accuracy: 1.0000
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.9031 - mae: 0.7584 - regression_accuracy: 0.6663 - val_loss: 0.5469 - val_mae: 0.7383 - val_regression_accuracy: 1.0000
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.7631 - mae: 0.6578 - regression_accuracy: 0.7597 - val_loss: 0.4740 - val_mae: 0.6871 - val_regression_accuracy: 1.0000
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.6740 - mae: 0.5932 - regression_accuracy: 0.7803 - val_loss: 0.4000 - val_mae: 0.6311 - val_regression_accuracy: 1.0000
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.6188 - mae: 0.5890 - regression_ac

In [34]:
model.save("modelo_lstm_temperatura.keras")